<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/Erweiterung_%C3%9Cbungsaufgabe_Auswahlproblem_Spediteur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auswahlproblem Spediteur

In [14]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [25]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [26]:
J = ['A1','A2','A3','A4','A5','A6'] # Menge der Aufträge

In [27]:
I = ["LKW1","LKW2","LKW3"] # Menge der LKW

## Entscheidungsvariablen

In [28]:
a=1
b=1

In [29]:
f"Summe von {a} und {b} ist: {a+b}"

'Summe von 1 und 1 ist: 2'

In [30]:
# Definition der Entscheidungsvariablen
x={}
for i in I:
  for j in J:
    print(f"Auftrag {j} auf {i}")
    x[i,j] = solver.BoolVar(f"{i,j}")
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Auftrag A1 auf LKW1
Auftrag A2 auf LKW1
Auftrag A3 auf LKW1
Auftrag A4 auf LKW1
Auftrag A5 auf LKW1
Auftrag A6 auf LKW1
Auftrag A1 auf LKW2
Auftrag A2 auf LKW2
Auftrag A3 auf LKW2
Auftrag A4 auf LKW2
Auftrag A5 auf LKW2
Auftrag A6 auf LKW2
Auftrag A1 auf LKW3
Auftrag A2 auf LKW3
Auftrag A3 auf LKW3
Auftrag A4 auf LKW3
Auftrag A5 auf LKW3
Auftrag A6 auf LKW3
Anzahl Entscheidungsvariablen = 18


## Parameter

In [32]:
db={} # Deckungsbeiträge der Aufträge
db['A1']=10
db['A2']=15
db['A3']=25
db['A4']=5
db['A5']=11
db['A6']=8

In [33]:
v={} # Volumen Aufträge der Aufträge
v['A1']=3
v['A2']=4.5
v['A3']=9.25
v['A4']=1.2
v['A5']=4
v['A6']=2.7

In [34]:
# Kapazitäten LKW
b = {}
b["LKW1"]=10
b["LKW2"]=8
b["LKW3"]=5

In [35]:
b

{'LKW1': 10, 'LKW2': 8, 'LKW3': 5}

## Zielfunktion

In [36]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(db[j]*x[i,j] for j in J for i in I))

## Nebenbedingungen

In [37]:
# LKW Kapazitäten
for i in I:
  solver.Add(sum(v[j]*x[i,j] for j in J)<=b[i])

# Einmalige Verladung
for j in J:
  solver.Add(sum(x[i,j] for i in I)<=1)

## Berechnung Lösung

In [40]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for i in I:
      for j in J:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 64.0
('LKW1', 'A3') = 1
('LKW2', 'A4') = 1
('LKW2', 'A5') = 1
('LKW2', 'A6') = 1
('LKW3', 'A2') = 1
